# 【総合演習】Antennaからの画像データのダウンロード  

### Selenium + BeautifulSoup + Requestsを組合せたスクレイピング

ここでは400もの提携メディアから流行のコンテンツを提供するキュレーションサービスであるAntenna( http://antenna.jp/ )を使って演習をします。

今後の商品開発に活かす為、どのようなノートPCがトレンドなのかを把握できるよう、検索ワード「ノートPC」で記事を検索し、検索結果のタイトルと詳細ページへのURL、画像ファイルを取得していきます。

### タスク1： Seleniumを用いてブラウザを起動し、Antennaのサイトを表示してください。その際に結果が確認しやすいよう、driver.maximize_window()を用いてブラウザを最大化して表示してください。

In [12]:
from selenium import webdriver

In [13]:
driver=webdriver.Chrome(r"C:\chromedriver-win64\chromedriver")
Antenna_url="https://antenna.jp/"
driver.get(Antenna_url)
driver.maximize_window()

### タスク2： Antennaのサイトの検索欄に「ノートPC」と入力し、検索結果を表示してください。

In [14]:
keyword_input=driver.find_element_by_id("search-input")
keyword_input.send_keys("ノートPC")
search_button=driver.find_element_by_id("search-button")
search_button.click()

### タスク3： 検索結果が全て表示されるよう、画面スクロールしてください。

In [15]:
import time

In [16]:
while True:
    last_height=driver.execute_script("return document.body.scrollHeight")
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    time.sleep(3)
    new_height=driver.execute_script("return document.body.scrollHeight")
    if new_height==last_height:
        break

### タスク4： 検索結果として表示された記事のタイトル、詳細ページへのURL、画像のURLを取得し、表示してください。<BR>また合わせて、最初の記事をNo.0として、記事に連番を振って表示してください。<BR>そして、画像のURLは、後からダウンロードできるよう、全てリストに格納してください。

In [21]:
from bs4 import BeautifulSoup

all_images=[]
soup=BeautifulSoup(driver.page_source,"html.parser")
elems_title=soup.select("div.title")
elems_url=soup.select("a.thumbnail-content")
elems_img=soup.select("div.image img")

i=0
for elem_title,elem_url,elem_img in zip(elems_title,elems_url,elems_img):
    all_images.append(elem_img)
    print("No."+str(i))
    print("タイトル："+elem_title.string) 
    print("URL："+elem_url["href"])
    print("画像："+elem_img["src"],end="\n\n")
    i+=1

No.0
タイトル：大人女子が求めるバッグ新登場！可愛い×収納力×超軽量=SYLPHY
URL：https://antenna.jp/articles/24597253
画像：https://staticx.antenna.jp/article_images/24597253_wide_2902ca24-cf6f-4be7-8ffa-efbc97937fc5_.jpeg

No.1
タイトル：コクヨの新作「水筒みたいなモバイルバッテリー」が便利そう！どこでも作業スペースにできそうだ
URL：https://antenna.jp/articles/24594307
画像：https://staticx.antenna.jp/article_images/24594307_wide_8719381b-932c-43b8-b477-0a0f29b2f9d5_.jpeg

No.2
タイトル：大容量フロントポケットのバックパックが便利すぎる！ビジネス、出張、撮影なんでもござれの収納力
URL：https://antenna.jp/articles/24590649
画像：https://staticx.antenna.jp/article_images/24590649_wide_94580f1a-86ff-41a5-b6e6-359a497302a9_.jpeg

No.3
タイトル：旅行・出張に持っていく充電器どうする問題、CIOの電源タップがまるっと解決する！
URL：https://antenna.jp/articles/24590936
画像：https://staticx.antenna.jp/article_images/24590936_wide_b6e01d3a-9a91-49b9-90a7-669164d1d109_.png

No.4
タイトル：アイデア満載！ 仕事や趣味が捗る！ 厳選「ガジェット＆周辺機器」7選【傑作＆大本命モノ決定版】
URL：https://antenna.jp/articles/24590333
画像：https://staticx.antenna.jp/article_images/24590333_wide_d9cf2367-c164-4ed9-9e63-6ec0a410f8c9_.jpeg

No.5
タイトル：「ポート足

### タスク5： リストに格納した画像のURLを元に、全ての画像をダウンロードしてください。画像のファイル名は、記事の連番としてください。

In [22]:
import os
import requests
import shutil

path=r"C:\Users\matsu\OneDrive\デスクトップ\移行\scraping\training_photo"

for index,image in enumerate(all_images):
    filename="image_"+str(index)+".jpg"
    image_path=os.path.join(path,filename)
    res=requests.get(image["src"],stream=True)
    with open(image_path,"wb") as file:
        shutil.copyfileobj(res.raw,file)


In [25]:
import csv
import datetime

csv_date=datetime.datetime.today().strftime("%Y%m%d")
csv_file_name="Antenna_"+csv_date+".csv"

f = open(csv_file_name,"w",encoding="cp932",errors="ignore")
writer=csv.writer(f, lineterminator="\n")
csv_header=["タイトル","URL"]
writer.writerow(csv_header)

i = 0
for elem_title,elem_url in zip(elems_title,elems_url):
    csvlist=[]
    csvlist.append(elem_title.string)
    csvlist.append(elem_url["href"])
    writer.writerow(csvlist)
    i += 1  # iを増加させる       
f.close()